# Training Block

In [5]:
import subprocess
import sys
import os
import glob
import random
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as T
from PIL import Image
from tqdm.auto import tqdm
from IPython.display import FileLink, display

# 1. INSTALL SMP
try:
    import segmentation_models_pytorch as smp
except ImportError:
    print("⬇️ Installing segmentation_models_pytorch...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "segmentation-models-pytorch"])
    import segmentation_models_pytorch as smp

# =============================================================================
# CONFIGURATION
# =============================================================================
CONFIG = {
    "ROOT_DIR": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/",
    "TEACHER_WEIGHTS": "weights/best_model.pth",  # Your current best B2 model
    "PSEUDO_DIR": "pseudo_masks",                 # Where we save fake labels
    
    # STUDENT CONFIG (The Upgrade)
    "STUDENT_ARCH": "mit_b3",        # UPGRADE: B2 -> B3 (Better Encoder)
    "INPUT_SIZE": (512, 512),        # Keep 512 for B3 memory safety
    "BATCH_SIZE": 8,                 # Lower batch size for B3 on T4
    "EPOCHS": 25,
    "LR": 2e-4,                      # Slightly lower LR for fine-tuning
    "CONFIDENCE_THR": 0.95,          # Only trust high-confidence predictions
    "FOREGROUND_IDS": [7100, 10000]
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Hardware: {torch.cuda.get_device_name(0)} x {torch.cuda.device_count()}")

# =============================================================================
# PART 1: THE TEACHER (Generator)
# =============================================================================
class TeacherModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Must match your saved B2 model structure
        self.model = smp.Unet(encoder_name="mit_b2", classes=1, decoder_use_batchnorm=True)

    def forward(self, x):
        return self.model(x)

def generate_pseudo_labels():
    print("👨‍🏫 Loading Teacher (mit_b2) to generate Pseudo-Labels...")
    
    if not os.path.exists(CONFIG["TEACHER_WEIGHTS"]):
        print("❌ Error: Teacher weights not found! Train the B2 model first.")
        return False

    # Load Teacher
    teacher = TeacherModel().to(device)
    state_dict = torch.load(CONFIG["TEACHER_WEIGHTS"], map_location=device)
    # Fix module prefix
    teacher.load_state_dict({k.replace("module.", ""): v for k, v in state_dict.items()})
    
    if torch.cuda.device_count() > 1: teacher = nn.DataParallel(teacher)
    teacher.eval()
    
    # Prepare Output Dir
    os.makedirs(CONFIG["PSEUDO_DIR"], exist_ok=True)
    
    # Get Test Images
    test_dir = os.path.join(CONFIG["ROOT_DIR"], "test_images_padded")
    if not os.path.exists(test_dir): test_dir = os.path.join(CONFIG["ROOT_DIR"], "test_images")
    test_files = sorted(glob.glob(os.path.join(test_dir, "*.*")))
    
    print(f"🔄 Generating labels for {len(test_files)} images...")
    
    count = 0
    with torch.no_grad():
        for img_path in tqdm(test_files):
            # Load & Preprocess
            image = Image.open(img_path).convert("RGB")
            orig_w, orig_h = image.size
            
            # Resize to training size for prediction (faster)
            input_tensor = T.functional.resize(image, CONFIG["INPUT_SIZE"])
            input_tensor = T.functional.to_tensor(input_tensor).unsqueeze(0).to(device)
            input_tensor = T.functional.normalize(input_tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            
            # Predict (Flip TTA)
            pred = torch.sigmoid(teacher(input_tensor))
            pred_flip = torch.sigmoid(teacher(torch.flip(input_tensor, dims=[3])))
            pred_avg = (pred + torch.flip(pred_flip, dims=[3])) / 2.0
            
            # Filter by Confidence (Soft Labeling)
            # We save the binary mask where confidence is high
            mask = (pred_avg > 0.5).float().cpu().numpy()[0,0]
            confidence = pred_avg.max().item()
            
            # Only save if model is somewhat sure (Optional: skip low conf images)
            # Here we save all, but you can add: if confidence < 0.8: continue
            
            # Resize back to original size for saving
            mask = cv2.resize(mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
            
            # Save as PNG
            save_name = os.path.basename(img_path).replace(".jpg", ".png")
            cv2.imwrite(os.path.join(CONFIG["PSEUDO_DIR"], save_name), mask * 255)
            count += 1
            
    print(f"✅ Generated {count} pseudo-masks in '{CONFIG['PSEUDO_DIR']}'")
    return True

# =============================================================================
# PART 2: THE STUDENT (Dataset & Model)
# =============================================================================
class PseudoDataset(Dataset):
    def __init__(self, root_dir, pseudo_dir, split='train'):
        self.split = split
        self.root_dir = root_dir
        
        # 1. Real Data
        self.real_img_dir = os.path.join(root_dir, 'train_images')
        self.real_mask_dir = os.path.join(root_dir, 'train_masks')
        self.real_images = sorted(glob.glob(os.path.join(self.real_img_dir, "*.*")))
        
        # 2. Pseudo Data (Only for training)
        self.pseudo_images = []
        if split == 'train' and os.path.exists(pseudo_dir):
            # We look for test images that have a matching mask in pseudo_dir
            test_img_dir = os.path.join(root_dir, 'test_images_padded')
            if not os.path.exists(test_img_dir): test_img_dir = os.path.join(root_dir, 'test_images')
            
            candidates = sorted(glob.glob(os.path.join(test_img_dir, "*.*")))
            for img_p in candidates:
                mask_name = os.path.basename(img_p).replace(".jpg", ".png")
                if os.path.exists(os.path.join(pseudo_dir, mask_name)):
                    self.pseudo_images.append((img_p, os.path.join(pseudo_dir, mask_name)))
            
            print(f"📚 Dataset: {len(self.real_images)} Real + {len(self.pseudo_images)} Pseudo")

        self.color_jitter = T.ColorJitter(0.2, 0.2, 0.2)

    def __len__(self):
        return len(self.real_images) + len(self.pseudo_images)

    def __getitem__(self, idx):
        # Determine if Real or Pseudo
        if idx < len(self.real_images):
            # REAL DATA
            img_path = self.real_images[idx]
            basename = os.path.splitext(os.path.basename(img_path))[0]
            
            # Try png/jpg mask
            mask_path = os.path.join(self.real_mask_dir, basename + ".png")
            if not os.path.exists(mask_path): mask_path = os.path.join(self.real_mask_dir, basename + ".jpg")
            
            image = Image.open(img_path).convert("RGB")
            if os.path.exists(mask_path):
                mask_src = np.array(Image.open(mask_path))
                mask = np.zeros_like(mask_src, dtype=np.uint8)
                for fg in CONFIG["FOREGROUND_IDS"]: mask[mask_src == fg] = 1
                mask = Image.fromarray(mask * 255)
            else:
                mask = Image.fromarray(np.zeros((image.size[1], image.size[0]), dtype=np.uint8))
        else:
            # PSEUDO DATA
            p_idx = idx - len(self.real_images)
            img_path, mask_path = self.pseudo_images[p_idx]
            image = Image.open(img_path).convert("RGB")
            mask = Image.open(mask_path).convert("L") # Already binary 0-255

        # Resize & Augment
        if self.split == 'train':
            # Resize if too small
            if image.size[0] < CONFIG["INPUT_SIZE"][0]:
                image = T.functional.resize(image, CONFIG["INPUT_SIZE"])
                mask = T.functional.resize(mask, CONFIG["INPUT_SIZE"], interpolation=T.InterpolationMode.NEAREST)
            
            # Random Crop
            i, j, h, w = T.RandomCrop.get_params(image, output_size=CONFIG["INPUT_SIZE"])
            image = T.functional.crop(image, i, j, h, w)
            mask = T.functional.crop(mask, i, j, h, w)
            
            # Augment
            if random.random() < 0.5:
                image = T.functional.hflip(image)
                mask = T.functional.hflip(mask)
            if random.random() < 0.3:
                 image = self.color_jitter(image)
        else:
            image = T.functional.resize(image, CONFIG["INPUT_SIZE"])
            mask = T.functional.resize(mask, CONFIG["INPUT_SIZE"], interpolation=T.InterpolationMode.NEAREST)

        # To Tensor
        image = T.functional.to_tensor(image)
        image = T.functional.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        mask = T.functional.to_tensor(mask)
        mask = (mask > 0.5).float()
        
        return image, mask

# =============================================================================
# PART 3: TRAINING LOOP (The Student)
# =============================================================================
def train_student():
    print(f"\n🎓 Training Student Model: {CONFIG['STUDENT_ARCH']}...")
    
    # 1. Dataset
    full_ds = PseudoDataset(CONFIG["ROOT_DIR"], CONFIG["PSEUDO_DIR"], split='train')
    
    # Validation only on REAL data (don't validate on pseudo labels!)
    train_size = int(0.9 * len(full_ds)) # Use more data for training
    val_size = len(full_ds) - train_size
    train_ds, val_ds = torch.utils.data.random_split(full_ds, [train_size, val_size])
    val_ds.dataset.split = 'val'
    
    train_loader = DataLoader(train_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=True, num_workers=4, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=False, num_workers=4)
    
    # 2. Student Model (MIT_B3 - The Upgrade)
    model = smp.Unet(
        encoder_name=CONFIG["STUDENT_ARCH"], 
        encoder_weights="imagenet", 
        in_channels=3, classes=1, decoder_use_batchnorm=True
    ).to(device)
    
    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)
    
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["LR"], weight_decay=1e-2)
    scaler = GradScaler()
    
    # Loss: Weighted heavily towards Dice for fine-tuning
    loss_fn = smp.losses.DiceLoss(mode='binary', from_logits=True)
    bce_fn = nn.BCEWithLogitsLoss()
    
    best_score = 0.0
    
    for epoch in range(CONFIG["EPOCHS"]):
        model.train()
        loop = tqdm(train_loader, desc=f"Ep {epoch+1}/{CONFIG['EPOCHS']}", leave=False)
        
        for imgs, masks in loop:
            imgs, masks = imgs.to(device), masks.to(device)
            optimizer.zero_grad()
            
            with autocast():
                preds = model(imgs)
                # 70% Dice (Shape), 30% BCE (Pixel accuracy)
                loss = 0.7 * loss_fn(preds, masks) + 0.3 * bce_fn(preds, masks)
                
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            loop.set_postfix(loss=loss.item())
            
        # Validate
        model.eval()
        val_iou = 0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(device), masks.to(device)
                with autocast(): preds = model(imgs)
                
                pred_mask = (torch.sigmoid(preds) > 0.5).float()
                inter = (pred_mask * masks).sum()
                union = pred_mask.sum() + masks.sum() - inter
                val_iou += (inter + 1e-6) / (union + 1e-6)
                
        avg_iou = val_iou / len(val_loader)
        print(f"Epoch {epoch+1} | Student Val IoU: {avg_iou:.4f}")
        
        if avg_iou > best_score:
            best_score = avg_iou
            # Save as 'best_student.pth' to differentiate
            torch.save(model.state_dict(), "weights/best_student_b3.pth")
            print(f"🏆 NEW STUDENT BEST! IoU: {best_score:.4f}")
            display(FileLink("weights/best_student_b3.pth"))

# =============================================================================
# EXECUTION
# =============================================================================
if __name__ == "__main__":
    # Step 1: Generate Labels
    success = generate_pseudo_labels()
    
    # Step 2: Train Student if labels exist
    if success:
        train_student()

🔥 Hardware: Tesla T4 x 2
👨‍🏫 Loading Teacher (mit_b2) to generate Pseudo-Labels...
🔄 Generating labels for 1002 images...


  0%|          | 0/1002 [00:00<?, ?it/s]

[ WARN:0@1.238] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


✅ Generated 1002 pseudo-masks in 'pseudo_masks'

🎓 Training Student Model: mit_b3...
📚 Dataset: 3174 Real + 1002 Pseudo


config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

...

Ep 24/25:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch 24 | Student Val IoU: 0.9038
🏆 NEW STUDENT BEST! IoU: 0.9038


/kaggle/working/weights/best_student_b3.pth

Ep 25/25:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch 25 | Student Val IoU: 0.9031


# Inference Pipeline

Change the paths section - 

DATA_ROOT - Your root directory path

WEIGHTS_PATH - Insert your model path here

TEST_DIR - Insert your test image path here


In [ ]:
import os
import glob
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
import torchvision.transforms as T
from IPython.display import FileLink, display
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp

# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Hardware Detected: {torch.cuda.get_device_name(0)} x {torch.cuda.device_count()}")

# Paths
DATA_ROOT = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/"

WEIGHTS_PATH = "weights/best_student_b3.pth"  

TEST_DIR = os.path.join(DATA_ROOT, "test_images_padded")

# Fallback to standard folder if padded doesn't exist
if not os.path.exists(TEST_DIR):
    print(f"⚠️ Padded folder not found. Falling back to standard test_images...")
    TEST_DIR = os.path.join(DATA_ROOT, "test_images")

# TTA Configuration
TTA_SCALES = [0.75, 1.0, 1.25]  # The "Holy Grail" scales

# ==========================================
# 2. MODEL ARCHITECTURE (Student mit_b3)
# ==========================================
def get_student_model():
    """
    Returns the exact model structure used in Student Training.
    Crucial: Must match 'mit_b3' if you trained the student with b3.
    """
    model = smp.Unet(
        encoder_name="mit_b3",       # <--- The Student Encoder
        encoder_weights=None,        # Weights are loaded later
        in_channels=3,
        classes=1,
        decoder_use_batchnorm=True,
    )
    return model

# ==========================================
# 3. HELPER FUNCTIONS
# ==========================================
def rle_encode(mask):
    """Encodes a binary mask to RLE format for Kaggle."""
    pixels = mask.flatten(order="F")
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def predict_multiscale_tta(model, image_tensor, scales=TTA_SCALES):
    """
    Runs inference at multiple scales with horizontal flip TTA.
    Returns the average probability map.
    """
    b, c, h, w = image_tensor.shape
    final_output = torch.zeros((b, 1, h, w), device=device)
    
    # Loop through scales
    for scale in scales:
        # 1. Resize Input (if needed)
        if scale != 1.0:
            new_h, new_w = int(h * scale), int(w * scale)
            # Ensure divisible by 32 (SegFormer requirement)
            new_h = int(np.ceil(new_h / 32) * 32)
            new_w = int(np.ceil(new_w / 32) * 32)
            
            input_scaled = F.interpolate(
                image_tensor, size=(new_h, new_w), mode='bilinear', align_corners=False
            )
        else:
            input_scaled = image_tensor

        # 2. Predict (Original + Flip)
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                # Forward Pass
                pred = torch.sigmoid(model(input_scaled))
                
                # Flip Pass (Horizontal)
                pred_flip = torch.sigmoid(model(torch.flip(input_scaled, dims=[3])))
                pred_flip = torch.flip(pred_flip, dims=[3])
        
        # 3. Average Flip & Original
        pred_avg = (pred + pred_flip) / 2.0
        
        # 4. Resize back to original target size (1.0x)
        if scale != 1.0:
            pred_avg = F.interpolate(
                pred_avg, size=(h, w), mode='bilinear', align_corners=False
            )
            
        final_output += pred_avg

    # Average over all scales
    final_output /= len(scales)
    return final_output

# ==========================================
# 4. MAIN INFERENCE ENGINE
# ==========================================
def run_inference():
    if not os.path.exists(WEIGHTS_PATH):
        print(f"❌ Critical Error: '{WEIGHTS_PATH}' not found!")
        print("   Did you run the Student Training cell successfully?")
        return

    # 1. Load Model
    print("⚙️ Loading Student Model (mit_b3)...")
    model = get_student_model()
    
    # 2. Load Weights (Robust Fix for 'module.' prefix)
    print(f"📖 Reading weights from {WEIGHTS_PATH}...")
    state_dict = torch.load(WEIGHTS_PATH, map_location=device)
    new_state_dict = {}
    for k, v in state_dict.items():
        # Strip 'module.' if it exists (from DataParallel training)
        k = k.replace("module.", "") 
        new_state_dict[k] = v
        
    model.load_state_dict(new_state_dict)
    model.to(device)
    model.eval()
    print("✅ Student Weights Loaded Successfully.")

    # 3. Process Images
    test_files = sorted(glob.glob(os.path.join(TEST_DIR, "*.*")))
    print(f"🚀 Processing {len(test_files)} images using Multi-Scale TTA {TTA_SCALES}...")
    
    results = []
    
    # Batch processing loop
    for img_path in tqdm(test_files):
        try:
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            
            # Load
            image = Image.open(img_path).convert("RGB")
            
            # Preprocess (Standard ImageNet Normalization)
            input_tensor = T.functional.to_tensor(image).unsqueeze(0).to(device)
            input_tensor = T.functional.normalize(input_tensor, 
                                                  mean=[0.485, 0.456, 0.406], 
                                                  std=[0.229, 0.224, 0.225])
            
            # INFERENCE: Multi-Scale TTA
            pred_mask = predict_multiscale_tta(model, input_tensor, scales=TTA_SCALES)
            
            # Binarize (Threshold 0.5)
            pred_mask_np = (pred_mask > 0.5).float().cpu().numpy().astype(np.uint8)[0, 0]
            
            # RLE Encode
            rle = rle_encode(pred_mask_np)
            results.append({'image_id': img_name, 'encoded_pixels': rle})
            
        except Exception as e:
            print(f"⚠️ Error on {img_name}: {e}")
            
    # 4. Save Submission
    df = pd.DataFrame(results)
    df.to_csv("submission.csv", index=False)
    
    print("\n✅ INFERENCE COMPLETE!")
    print(f"📊 Processed {len(results)} images.")
    print("⬇️ CLICK BELOW TO DOWNLOAD YOUR SUBMISSION:")
    display(FileLink("submission.csv"))

if __name__ == "__main__":
    run_inference()

🔥 Hardware Detected: Tesla T4 x 2
⚙️ Loading Student Model (mit_b3)...
📖 Reading weights from weights/best_student_b3.pth...
✅ Student Weights Loaded Successfully.
🚀 Processing 1002 images using Multi-Scale TTA [0.75, 1.0, 1.25]...


  0%|          | 0/1002 [00:00<?, ?it/s]

/tmp/ipykernel_55/2353206082.py:86: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



✅ INFERENCE COMPLETE!
📊 Processed 1002 images.
⬇️ CLICK BELOW TO DOWNLOAD YOUR SUBMISSION:


/kaggle/working/submission.csv